# Second Harmonic Generation
---

### **Overview**

The simulation models Type-0 SHG, where two photons of the same frequency $(\omega)$ interact in a nonlinear KTP crystal to generate a photon at twice the frequency $(2\omega).$ All fields are polarized along the crystal’s z-axis, leveraging the highest nonlinear coefficient. **Quasi-phase matching (QPM)** is achieved using a poling period $(\Lambda)$. The goal is to compute the second-harmonic field $A_{2\omega}(\omega)$ at the output of a crystal of length L, accounting for optical properties, dispersion, and phase-matching conditions.

### **Required Parameters**

The key parameters, extracted from the provided information, are organized into categories:

1. **Crystal Properties (KTP)**
    - **Refractive Index $(n_z(\lambda))$**: Calculated using the Sellmeier equation:
    
    $$
    n_{z}(\lambda) = \sqrt{a_{k} + \frac{b_{k}}{1 - \frac{c_{k}}{\lambda^{2}}} 
                      + \frac{d_{k}}{1 - \frac{e_{k}}{\lambda^{2}}} 
                      - f_{k}\lambda^{2}}
    $$
    
    where the coefficients read $a_k = 2.12725, b_k = 1.18431, c_k = 5.14852 \times 10^2,dk = 0.6603, e_k = 100.00507$  and $f_k = 9.68956 \times 10^3$  and where λ is in μm.
    
    - **Group Velocity Index $v_{g,z}(\lambda)$**:
    
    $$
    ng,z(λ)=nz(λ)−\lambda\frac{\lambda n_z}{\partial \lambda}
    $$
    
    with $c = 3 \times 10^8 \, \text{m/s}$.
    
    - **Nonlinear Susceptibility $(\chi^{(2)})$**: Second-order nonlinear coefficient (specific value not provided, but critical for the coupling term).
    
    1. **Wave Parameters**
        - **Fundamental Wavelength $(\lambda_0)$**: 795 nm ( $\omega_0 =\lambda \frac{2\pi c}{\lambda_0}$).
        - **Second-Harmonic Wavelength**: 397.5 nm $(2\omega_0)$.
        - **Fundamental Field $(A_\omega(\omega))$**: Input pulse spectrum (shape not specified, e.g., Gaussian).
        - **Crystal Length (L)**: Propagation length.
    
    1. **Quasi-Phase Matching (QPM)**
        - **Poling Period $(\Lambda)$**:
    
    $$
    \Lambda = \frac{\pi c}{\omega_{0}} \left( n_{2\omega}(2\omega_{0}) - n_{\omega}(\omega_{0}) \right) = 3.19 \mu m
    $$
    
    Calculated for $\lambda_0 = 795 \, \text{nm}$ and $\lambda_{2\omega} = 397.5 \, \text{nm}$.
    
    1. **Phase-Matching Function $(\Phi(\omega))$**
        - **Phase-Matching Function**:
        
        $$
        \Phi(\omega', \omega) = \text{sinc} \left( \frac{\Delta k(\omega, \omega') L}{2} \right)
        $$
        
    2. Other Parameters
        - **Coupling Coefficient**:
        
        $$
        \frac{i \omega_0 \chi^{(2)} L}{n_{2\omega} c}
        $$
        
        - **Self-Convolution Term $(F(\omega))$**: Autoconvolution of the fundamental field
            
            $$
            F(\omega) = \int_{\mathbb{R}} A_\omega(\omega') A_\omega(\omega - \omega') \, d\omega'
            $$
            
        - **Factor $G_m$**: Not explicitly defined.


### **Simulation Steps**

The steps for simulating Type-0 SHG in KTP are as follows

1. **Set Up Initial Parameters**
    1. Define wavelengths:   $λ_0=795 \text{ nm}, λ_{2\omega}=397.5\text{ nm}$.
    2. Calculate $\omega_0 = \frac{2\pi c}{\lambda_0}$.
    3. Specify the fundamental field spectrum $A_\omega(\omega)$ (e.g., Gaussian pulse).
    4. Define crystal length L and $\chi^{(2)}$ value.
    5. Set $\Lambda = 3.19 \, \mu m$ for QPM.
    
2. **Calculate Crystal Optical Properties**
    1. Use the Sellmeier equation to compute $n_z(\lambda)$ at $\lambda = 795 \, \text{nm}$and $\lambda = 397.5 \, \text{nm}$.
    2. Derive $\frac{\partial n_z}{\partial \lambda}$ analytically to calculate $n_{g,z}(\lambda)$.
        
        $$
        ng,z(λ)=nz(λ)−\lambda\frac{\lambda n_z}{\partial \lambda}
        $$
        
    3. Compute group velocities for $\lambda = 795 \, \text{nm}$and $\lambda = 397.5 \, \text{nm}$.
    
3. **Calculate Phase Mismatch**
    1. Evaluate $\Delta k(\omega)$
        
        $$
        \Delta k(\omega) = \left( \frac{1}{v_g(2\omega_0)} - \frac{1}{v_g(\omega_0)} \right) (\omega - 2\omega_0)
        $$
        
    2. Verify $\Delta k_{0th} = 0$ for QPM with $\Lambda = 3.19 \, \mu m$.
    
4. Compute Phase-Matching Function Φ(ω′,ω).
5. Calculate Fundamental Field auto convolution F(ω).
6. **Calculate Second-Harmonic Field:**
    1.  ****Evaluate the equation
        
        $$
        A_{2\omega}(\omega) = \left( \frac{i \omega_0 \chi^{(2)} L}{n_{2\omega} c} \right) F(\omega) G_m \Phi'(\omega)
        $$
        
    2. Obtain the spectrum $A_{2\omega}(\omega)$.




In [1]:
import sys
sys.path.append('../..') # Asegúrate de que Python pueda encontrar el paquete src

# importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

from src.physics.pulses import GaussianPulse1D, format_value
from src.physics.crystals import KTPCrystal, KTPCrystal_Kato

### Set up initial parameters
---

In [2]:
# Gaussian input pulse parameters
# Input beam parameters
input_width_wl = 44e-9             # (m) wl: wavelength
fundamental_wavelength = 795e-9    # (m)

input_pulse = GaussianPulse1D(x0=fundamental_wavelength, FWHM=input_width_wl, steps=1000)
wavelenght, y_wavelenght = input_pulse.generate_intensity()
sigma_wavelength = input_pulse.standard_deviation()  # (m) | 1.87e-08 m


# Parameters in frequency
c = 299792458  # (m/s)
fundamental_frequency = (2*np.pi*c) / fundamental_wavelength  # (Hz = 1/s) | 23.69 PHz
sigma_frequency = (2*np.pi*c) / (fundamental_wavelength**2) * sigma_wavelength  # (Hz) | 55.69 PHz

input_pulse_omega = GaussianPulse1D(x0=fundamental_frequency, steps=1000, std=sigma_frequency)
omega, A_omega = input_pulse_omega.generate_intensity()

print("Parameters of the Gaussian pulse in wavelength domain:")
print("-"*60)
# Obtener el diccionario con los parámetros del pulso
pulse_dict = input_pulse.as_dict()
for key, value in pulse_dict.items():
    formatted_value = format_value(value)
    print(f"{key.replace('_', ' ').title()}: {formatted_value}")
    
print("\n")
print("Parameters of the Gaussian pulse in frequency domain:")
print("-"*60)
# Obtener el diccionario con los parámetros del pulso
pulse_dict_omega = input_pulse_omega.as_dict()
for key, value in pulse_dict_omega.items():
    formatted_value = format_value(value)
    print(f"{key.replace('_', ' ').title()}: {formatted_value}")

Parameters of the Gaussian pulse in wavelength domain:
------------------------------------------------------------
Center: 7.95e-07
Fwhm: 4.40e-08
Sigma: 1.87e-08
Steps: 1000
Times Std: 5
Computed Fwhm: 4.40e-08


Parameters of the Gaussian pulse in frequency domain:
------------------------------------------------------------
Center: 2.37e+15
Fwhm: None
Sigma: 5.57e+13
Steps: 1000
Times Std: 5
Computed Fwhm: 1.31e+14


### Sellmeir refractive index
---

In [3]:
ktp = KTPCrystal()

# Calcular índice para una longitud de onda y dirección
wavelength_micrometers = 0.532 # 532 nm
index_nz = ktp.refractive_index(wavelength_micrometers, axis='nz')
print(f"Refractive index at {wavelength_micrometers} um (nz): {index_nz:.6f}") # Formato para mejor lectura
wavelength_micrometers = 1.064 # 1064 nm
index_nx = ktp.refractive_index(wavelength_micrometers, axis='nx')
print(f"Refractive index at {wavelength_micrometers} um (nx): {index_nx:.6f}")

Refractive index at 0.532 um (nz): 1.886821
Refractive index at 1.064 um (nx): 1.739908


In [4]:
ktp_kato = KTPCrystal_Kato()

# Calcular índice para una longitud de onda y dirección
wavelength_micrometers = 0.532 # 532 nm
index_nz = ktp_kato.refractive_index(wavelength_micrometers, axis='nz')
print(f"Refractive index at {wavelength_micrometers} um (nz): {index_nz:.6f}") # Formato para mejor lectura
wavelength_micrometers = 1.064 # 1064 nm
index_nx = ktp_kato.refractive_index(wavelength_micrometers, axis='nx')
print(f"Refractive index at {wavelength_micrometers} um (nx): {index_nx:.6f}")

Refractive index at 0.532 um (nz): 1.888651
Refractive index at 1.064 um (nx): 1.737926
